# Export feature embedding file
在分布式训练过程中，特征的embedding数据会以二进制的格式序列化到hdfs，在online serving时需要反序列化，后续我们会支持训练过程中直接导出文本格式的embedding数据，现在提供一个工具，将二进制数据反序列化为文本数据。

### 1. 二进制格式转文本格式
- 编译转换工具```emb_converter```

  在```tensornet```目录下编译```emb_converter```

In [34]:
!cd /da1/s/yaolei/tensornet && ./bazel build -c opt //core:emb_converter

Loading: 
Loading: 0 packages loaded
Analyzing: target //core:emb_converter (1 packages loaded, 0 targets configure\
d)
Analyzing: target //core:emb_converter (1 packages loaded, 4 targets configure\
d)
INFO: Analyzed target //core:emb_converter (1 packages loaded, 4 targets configured).

INFO: Found 1 target...

[0 / 3] [Prepa] BazelWorkspaceStatusAction stable-status.txt
[4 / 7] 2 actions running
    Compiling core/utility/file_io.cc; 0s processwrapper-sandbox
    Compiling core/tools/emb_converter.cc; 0s processwrapper-sandbox
[4 / 7] 2 actions running
    Compiling core/utility/file_io.cc; 1s processwrapper-sandbox
    Compiling core/tools/emb_converter.cc; 1s processwrapper-sandbox
[4 / 7] 2 actions running
    Compiling core/utility/file_io.cc; 2s processwrapper-sandbox
    Compiling core/tools/emb_converter.cc; 2s processwrapper-sandbox
[5 / 7] Compiling core/tools/emb_converter.cc; 2s processwrapper-sandbox
[5 / 7] Compiling core/tools/emb_converter.cc; 3s processwrapper-sandbo

- 执行emb_converter

In [24]:
!/da1/s/yaolei/tensornet/bazel-bin/core/emb_converter /tmp/model/2020-05-11/sparse_table/ /tmp/model/sparse_text AdaGrad

2020-08-26 14:38:30.988784: I core/tools/emb_converter.cc:82] file [/tmp/model/2020-05-11/sparse_table//0/rank_0/sparse_block_0] processed.
2020-08-26 14:38:30.988914: I core/tools/emb_converter.cc:82] file [/tmp/model/2020-05-11/sparse_table//0/rank_0/sparse_block_1] processed.
2020-08-26 14:38:30.988965: I core/tools/emb_converter.cc:82] file [/tmp/model/2020-05-11/sparse_table//0/rank_0/sparse_block_2] processed.
2020-08-26 14:38:30.989013: I core/tools/emb_converter.cc:82] file [/tmp/model/2020-05-11/sparse_table//0/rank_0/sparse_block_3] processed.
2020-08-26 14:38:30.989059: I core/tools/emb_converter.cc:82] file [/tmp/model/2020-05-11/sparse_table//0/rank_0/sparse_block_4] processed.
2020-08-26 14:38:30.989114: I core/tools/emb_converter.cc:82] file [/tmp/model/2020-05-11/sparse_table//0/rank_0/sparse_block_5] processed.
2020-08-26 14:38:30.989160: I core/tools/emb_converter.cc:82] file [/tmp/model/2020-05-11/sparse_table//0/rank_0/sparse_block_6] processed.
2020-08-26 14:38:30.

- 生成数据格式
  
  第一列为feature sign， 第二列为table_handle，后面的列为多维的embedding。

In [29]:
!tail -n 10 /tmp/model/sparse_text

664727995372584705	1	-0.025697	0.080912	-0.002423	0.013580	0.053367	0.017855	0.013988	0.002631
8658432068820613603	1	-0.004544	-0.012241	0.019810	-0.034679	0.028699	-0.020262	-0.063680	0.016947
7797335242211775142	1	0.057663	0.004491	-0.021192	0.034264	0.061628	-0.026370	-0.016498	0.027886
7391248522983175429	1	-0.007909	0.057782	0.018526	-0.008813	-0.063427	-0.048736	-0.022533	-0.024117
6443217198606683085	1	0.032003	-0.050358	0.021636	0.016701	-0.060406	-0.053007	-0.013675	0.048079
5194023452875547590	1	0.023324	0.003480	-0.009819	-0.042841	0.006762	0.005401	0.010376	-0.016773
5178955677997930731	1	-0.073632	0.021773	-0.019133	-0.028759	0.002195	0.024881	0.065171	-0.002595
5255158498343325654	1	-0.056173	-0.009087	0.043519	-0.054567	0.035521	-0.046024	0.002395	-0.036604
2951414286115464033	1	-0.072051	0.037498	-0.018196	-0.025876	0.045073	-0.007181	0.042396	0.061262
336571397910699740	1	0.043650	-0.019351	0.013762	0.023796	-0.051640	0.009684	0.076476	-0.017673


### 2. 相同feature embedding 合并
- 训练时，每一个```tn.layers.EmbeddingFeatures```都有独立的embedding，在wide&deep的demo中，每个feature会有两个不同的embedding，一个1维，一个8维，为了减少线上embedding lookup次数，下面脚本将相同feature的embedding按照table_handle递增顺序合并为一行

In [27]:
!cd /da1/s/yaolei/tensornet/examples/online_serving/ && \
 sort -nk1,2  /tmp/model/sparse_text | python emb_converter_reduce.py > /tmp/model/sparse_merged.txt

- 生成数据格式

  如下，第一列为feature sign，后面为9维度的embedding

In [30]:
!tail -n 10 /tmp/model/sparse_merged.txt

8621754627708332486	-0.020929	0.017268	0.011393	0.050349	-0.045138	0.013934	0.034815	-0.108272	-0.003104
8658432068820613603	-0.090976	-0.004544	-0.012241	0.019810	-0.034679	0.028699	-0.020262	-0.063680	0.016947
8739948393308632152	0.012600	-0.045059	0.003665	-0.001669	0.036731	0.047757	-0.003013	-0.010163	0.045259
8828880780756261625	0.073003	0.055755	0.022523	0.009402	-0.010848	0.035887	-0.070689	0.019089	0.025007
8895687830835085643	-0.203445	-0.016662	-0.013364	-0.022973	0.016830	-0.088185	0.002384	-0.011923	0.007559
8958114852440850551	-0.135199	-0.054110	-0.037400	-0.025072	0.049798	0.000475	-0.021564	-0.034338	-0.011842
8993353759708287499	0.017241	-0.039746	-0.001939	-0.055951	-0.021930	-0.012993	0.031334	-0.017015	-0.010426
9036706772189321662	0.067262	-0.009823	0.073938	-0.032093	0.029376	-0.037094	-0.016799	0.003812	-0.027969
9072674328386204684	-0.013450	0.011828	-0.005519	-0.043596	0.010915	-0.041561	0.050177	-0.066181	-0.075772
9116662905595839563	0.032742	0.005541	-0.001

此demo只为演示功能，生产环境可以使用MapReduce来替代`sort -nk1,2`。